In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()

# Spark SQL

In [2]:
df = spark.sql('''select "OK" as Status''')
df.show()

+------+
|Status|
+------+
|    OK|
+------+



# Import data

In [12]:
df = spark.read.csv('./cereal.csv', sep=',', inferSchema=True, header=True)

print(f'Lines: {df.count()}')
df.show(10)

Lines: 77
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple

In [4]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



# Manipulate date with Spark SQL

In [7]:
df.createOrReplaceTempView('cereal_sql')

In [8]:
df_cereal = spark.sql('''SELECT * FROM cereal_sql''')
df_cereal.show(5)

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
+---------------

In [14]:
filter = spark.sql('''SELECT * FROM cereal_sql WHERE type = "C"''')
print(f'Lines: {filter.count()}')
filter.show(5)

Lines: 74
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
+-----

# Counting lines as new column

In [15]:
filter = spark.sql('''SELECT COUNT(*) AS total FROM cereal_sql WHERE mfr = "K"''')
filter.show()

+-----+
|total|
+-----+
|   23|
+-----+



# Variações do comando Select

In [16]:
# Colunas específicas
cereal = spark.sql('''SELECT name,type,mfr FROM cereal_sql ''')
cereal.show(5)

+--------------------+----+---+
|                name|type|mfr|
+--------------------+----+---+
|           100% Bran|   C|  N|
|   100% Natural Bran|   C|  Q|
|            All-Bran|   C|  K|
|All-Bran with Ext...|   C|  K|
|      Almond Delight|   C|  R|
+--------------------+----+---+
only showing top 5 rows



In [18]:
# Valores distintos
cereal = spark.sql('''SELECT type,mfr FROM cereal_sql ''')
print(f'Lines: {cereal.count()}')
cereal.show(5)

cereal = spark.sql('''SELECT DISTINCT type,mfr FROM cereal_sql ''')
print(f'Lines: {cereal.count()}')
cereal.show(5)

Lines: 77
+----+---+
|type|mfr|
+----+---+
|   C|  N|
|   C|  Q|
|   C|  K|
|   C|  K|
|   C|  R|
+----+---+
only showing top 5 rows

Lines: 9
+----+---+
|type|mfr|
+----+---+
|   C|  P|
|   C|  Q|
|   C|  N|
|   H|  Q|
|   C|  R|
+----+---+
only showing top 5 rows



# Comando Where

In [21]:
cereal = spark.sql('''SELECT DISTINCT type,mfr FROM cereal_sql WHERE type = "C"''')
print(f'Lines: {cereal.count()}')
cereal.show(5)

Lines: 6
+----+---+
|type|mfr|
+----+---+
|   C|  P|
|   C|  Q|
|   C|  N|
|   C|  R|
|   C|  G|
+----+---+
only showing top 5 rows



In [22]:
# Dupla condição
cereal = spark.sql('''SELECT * FROM cereal_sql WHERE mfr = "K" AND calories >= 100''')
print(f'Lines: {cereal.count()}')
cereal.show(5)

Lines: 19
+------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|              name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|       Apple Jacks|  K|   C|     110|      2|  0|   125|  1.0| 11.0|    14|    30|      25|    2|   1.0| 1.0|33.174094|
|       Corn Flakes|  K|   C|     100|      2|  0|   290|  1.0| 21.0|     2|    35|      25|    1|   1.0| 1.0|45.863324|
|         Corn Pops|  K|   C|     110|      1|  0|    90|  1.0| 13.0|    12|    20|      25|    2|   1.0| 1.0|35.782791|
|Cracklin' Oat Bran|  K|   C|     110|      3|  3|   140|  4.0| 10.0|     7|   160|      25|    3|   1.0| 0.5|40.448772|
|           Crispix|  K|   C|     110|      2|  0|   220|  1.0| 21.0|     3|    30|      25|    3|   1.0| 1.0|46.895644|
+------------------+--

# Groupby

In [27]:
# Agrupamento simples
cereal = spark.sql(''' SELECT mfr,type,COUNT(*) AS count FROM cereal_sql GROUP BY mfr,type''')
cereal.show()

+---+----+-----+
|mfr|type|count|
+---+----+-----+
|  A|   H|    1|
|  P|   C|    9|
|  K|   C|   23|
|  G|   C|   22|
|  Q|   C|    7|
|  R|   C|    8|
|  Q|   H|    1|
|  N|   H|    1|
|  N|   C|    5|
+---+----+-----+



In [28]:
# Agrupamento com mais de uma agregação
cereal = spark.sql(''' SELECT mfr,type,COUNT(*) AS count, SUM(calories) as total_calories FROM cereal_sql GROUP BY mfr,type''')
cereal.show()

+---+----+-----+--------------+
|mfr|type|count|total_calories|
+---+----+-----+--------------+
|  A|   H|    1|           100|
|  P|   C|    9|           980|
|  K|   C|   23|          2500|
|  G|   C|   22|          2450|
|  Q|   C|    7|           660|
|  R|   C|    8|           920|
|  Q|   H|    1|           100|
|  N|   H|    1|           100|
|  N|   C|    5|           420|
+---+----+-----+--------------+



In [29]:
# Agrupamento com mais de uma agregação
cereal = spark.sql(''' SELECT \
                           mfr, type, COUNT(*) AS count, SUM(calories) as total_calories \
                       FROM cereal_sql\
                       GROUP BY mfr,type''')
cereal.show()

+---+----+-----+--------------+
|mfr|type|count|total_calories|
+---+----+-----+--------------+
|  A|   H|    1|           100|
|  P|   C|    9|           980|
|  K|   C|   23|          2500|
|  G|   C|   22|          2450|
|  Q|   C|    7|           660|
|  R|   C|    8|           920|
|  Q|   H|    1|           100|
|  N|   H|    1|           100|
|  N|   C|    5|           420|
+---+----+-----+--------------+



# CASE WHEN

In [30]:
cereal = spark.sql(''' SELECT DISTINCT\
                           type \
                       FROM cereal_sql''')
cereal.show()

+----+
|type|
+----+
|   C|
|   H|
+----+



In [34]:
cereal = spark.sql(''' SELECT\
                           mfr, type, (case when type = "C" then "tipo C" else "tipo H" end) as category, count(*) as count \
                       FROM cereal_sql\
                       GROUP BY mfr, type''')
cereal.show()

+---+----+--------+-----+
|mfr|type|category|count|
+---+----+--------+-----+
|  A|   H|  tipo H|    1|
|  P|   C|  tipo C|    9|
|  K|   C|  tipo C|   23|
|  G|   C|  tipo C|   22|
|  Q|   C|  tipo C|    7|
|  R|   C|  tipo C|    8|
|  Q|   H|  tipo H|    1|
|  N|   H|  tipo H|    1|
|  N|   C|  tipo C|    5|
+---+----+--------+-----+



In [48]:
# Células comentadas em SQL
cereal = spark.sql(''' SELECT
                           mfr, type,
                           (case
                                when type = "C" then "tipo C"
                                --when type = "H" then "tipo B"
                                --else "tipo H"
                           end) as category,
                           count(*) as count 
                       FROM cereal_sql
                       GROUP BY mfr, type''')
cereal.show()

+---+----+--------+-----+
|mfr|type|category|count|
+---+----+--------+-----+
|  A|   H|    null|    1|
|  P|   C|  tipo C|    9|
|  K|   C|  tipo C|   23|
|  G|   C|  tipo C|   22|
|  Q|   C|  tipo C|    7|
|  R|   C|  tipo C|    8|
|  Q|   H|    null|    1|
|  N|   H|    null|    1|
|  N|   C|  tipo C|    5|
+---+----+--------+-----+



# Consultas avançadas

In [55]:
cereal = spark.sql('''  SELECT mfr, type,
                                sum(calories) as sum_calories,
                                min(calories) as min_calories,
                                max(calories) as max_calories,
                                avg(calories) as avg_calories,
                                count(name) as count
                        FROM cereal_sql
                        GROUP BY mfr,type
                        ORDER BY count DESC

''')

cereal.show()

+---+----+------------+------------+------------+------------------+-----+
|mfr|type|sum_calories|min_calories|max_calories|      avg_calories|count|
+---+----+------------+------------+------------+------------------+-----+
|  K|   C|        2500|          50|         160|108.69565217391305|   23|
|  G|   C|        2450|         100|         140|111.36363636363636|   22|
|  P|   C|         980|          90|         120|108.88888888888889|    9|
|  R|   C|         920|          90|         150|             115.0|    8|
|  Q|   C|         660|          50|         120| 94.28571428571429|    7|
|  N|   C|         420|          70|          90|              84.0|    5|
|  A|   H|         100|         100|         100|             100.0|    1|
|  Q|   H|         100|         100|         100|             100.0|    1|
|  N|   H|         100|         100|         100|             100.0|    1|
+---+----+------------+------------+------------+------------------+-----+

